In [1]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
df = pd.read_csv('./results_v2.csv',index_col=0)

all_case = df['Case_Study'].unique()

app.layout = html.Div([

    html.Div([
        html.Div([
            html.H3(children='Results Analysis for WaterTAP3'),
        ], style={
            'padding': '0px 10px 0px 10px'}),
    ], style={
            'width': '70%', 
            'textAlign': 'left',
            'backgroundColor': 'rgb(250, 250, 250)',
            'padding': '10px 10px 10px 10px'}),

    html.Div([
        html.Div([
            html.Label('Select study case:'),
            dcc.Dropdown(
                id='crossfilter-case',
                options=[{'label': i, 'value': i} for i in all_case],
                value = 'Carlsbad',
            ),
        ], style={
            'width': '47%', 
            'display': 'inline-block',
            'padding': '10px 10px'}),

        html.Div([    
            html.Label('Avaliable scenarios:'),
            dcc.Dropdown(
                id='crossfilter-scenario',
                value = 'Baseline',
            ),
        ], style={
            'width': '47%', 
            'display': 'inline-block',
            'padding': '10px 10px'}),

        html.Div([
            html.Label('Outputs options:'),
            dcc.Checklist(
                id='crossfilter-graph-type',
                options=[
                    {'label': 'Cost of Water Breakdown', 'value': 'Cost'},
                    {'label': 'Total Water Intensity', 'value': 'Waterflow'},
                    {'label': 'Energy Intensity', 'value': 'Energy'},
                    {'label': 'Environmental Impacts', 'value': 'Impact'},
                ],
                value=['Cost'],
                labelStyle={'display': 'inline-flex'},
            ),        
        ], style={ 
            'display': 'inline-block',
            'padding': '0px 10px'}),
    ], style={
        'width': '70%',
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '0px 10px 10px 10px'}),

    html.Div([
        dcc.Graph(id='figure-bar')
    ], style={
        'width': '70%', 
        'padding': '20px 10px'}),

    html.Div([
        html.Label('Avaliable unit processes:'),
        dcc.Dropdown(
            id='crossfilter-unit-process',
            value = ['swoi','tri_media_filtration','cf'],
            multi=True,
        ),
    ], style={
        'width': '53%', 
        'padding': '5px 55px'}),
    
    html.Div([
        html.Label('Avaliable variables:'),
        dcc.Dropdown(
            id='crossfilter-variable',
            value = ['fixed_cap_inv','total_up_cost','inlet_flow'],
            multi=True,
        ),
    ], style={
        'width': '53%', 
        'padding': '5px 55px'}),

], style={
    'margin-top': '20px', 
    'margin-left': '50px'})


@app.callback(
    Output('crossfilter-scenario', 'options'),
    [Input('crossfilter-case', 'value')])
def set_scenario_options(selected_case):
    ddf = df[(df['Case_Study']==selected_case)]
    return [{'label': i, 'value': i} for i in ddf['Scenario'].unique()]

@app.callback(
    Output('crossfilter-unit-process', 'options'),
    [Input('crossfilter-scenario', 'value'),
     Input('crossfilter-case', 'value'),
     Input('crossfilter-graph-type', 'value')])
def set_unit_process_options(selected_scenario, selected_case, selected_graph_type):
    
    ddf2 = df[(df['Case_Study']==selected_case)&(df['Scenario']==selected_scenario)]

    if selected_graph_type == ['Cost']:
        ddf2_new = ddf2[ddf2['Unit']=='$MM']
    elif selected_graph_type == ['Waterflow']:
        ddf2_new = ddf2[ddf2['Unit']=='m3/s']
    else: 
        ddf2_new = ddf2.copy()

    return [{'label': i, 'value': i} for i in ddf2_new['Unit_Process'].unique()]

@app.callback(
    Output('crossfilter-variable', 'options'),
    [Input('crossfilter-scenario', 'value'),
     Input('crossfilter-case', 'value'),
     Input('crossfilter-graph-type', 'value')])
def set_variable_options(selected_scenario, selected_case, selected_graph_type):
    ddf2 = df[(df['Case_Study']==selected_case)&(df['Scenario']==selected_scenario)]

    if selected_graph_type == ['Cost']:
        ddf2_new = ddf2[ddf2['Unit']=='$MM']
    elif selected_graph_type == ['Waterflow']:
        ddf2_new = ddf2[ddf2['Unit']=='m3/s']
    else: 
        ddf2_new = ddf2.copy()

    return [{'label': i, 'value': i} for i in ddf2_new['Variable'].unique()]

@app.callback(
    Output('figure-bar', 'figure'),
    [Input('crossfilter-unit-process', 'value'),
     Input('crossfilter-variable', 'value'),
     Input('crossfilter-scenario', 'value'),
     Input('crossfilter-case', 'value'),
     Input('crossfilter-graph-type', 'value')])
def update_graph(selected_unit_process, selected_variable,
                    selected_scenario, selected_case, selected_graph_type):
    
    ddf3 = df[(df['Case_Study']==selected_case)
                &(df['Scenario']==selected_scenario)
                &(df['Unit_Process'].isin(selected_unit_process))
                &(df['Variable'].isin(selected_variable))]
    
    ddf4_c = ddf3[ddf3['Unit']=='$MM']
    ddf4_w = ddf3[ddf3['Unit']=='m3/s']

    if selected_graph_type == ['Cost']:

        fig = make_subplots()
        fig = px.bar(ddf4_c, x='Unit_Process', y='Value', color='Variable')
        fig.update_xaxes(title='Unit_Process')
        fig.update_yaxes(title='Cost ($MM)', type='linear')
        fig.update_layout(title_text="Cost Results Analysis")

    elif selected_graph_type == ['Waterflow']:
        
        fig = make_subplots()
        fig = px.bar(ddf4_w, x='Unit_Process', y='Value', color='Variable')
        fig.update_xaxes(title='Unit_Process')
        fig.update_yaxes(title='WaterFlow (m3/s)', type='linear')
        fig.update_layout(title_text="Waterflow Results Analysis")
    
    else:

        fig = make_subplots(specs=[[{"secondary_y": True}]])

        for var, group in ddf4_c.groupby('Variable'):
            fig.add_trace(go.Bar(x=group['Unit_Process'], y=group['Value'], width=0.4, name=var, offset=-0.2))
        
        for var, group in ddf4_w.groupby('Variable'):
            fig.add_trace(go.Bar(x=group['Unit_Process'], y=group['Value'], width=0.4, name=var, offset=0.2),secondary_y=True,)
        
        fig.update_xaxes(title_text="Unit_Process")
        fig.update_yaxes(title_text="Cost ($MM)", secondary_y=False)
        fig.update_yaxes(title_text="WaterFlow (m3/s)", secondary_y=True)
        fig.update_layout(barmode='stack', title_text="Combined Results Analysis")

    fig.update_layout(margin={'l': 40, 'b': 30, 't': 30, 'r': 0}, hovermode='closest')

    return fig


if __name__ == '__main__':
    app.run_server(debug=True,port=8051)

Dash app running on http://127.0.0.1:8051/
